[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/keras-team/autokeras/blob/master/docs/templates/tutorial/image_classification.ipynb)

In [ ]:
pip install autokeras

In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

## A Simple Example

### Load MNIST dataset

In [2]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape) # (60000, 28, 28)
print(y_train.shape) # (60000,)
print(y_train[:3]) # array([7, 2, 1], dtype=uint8)

(60000, 28, 28)
(60000,)
[5 0 4]


### Run the ImageClassifier

In [4]:
import autokeras as ak

# Initialize the image classifier.
clf = ak.ImageClassifier(max_trials=2) # It tries 10 different models.
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=1, verbose=2)


Train for 1500 steps, validate for 375 steps
1500/1500 - 15s - loss: 0.1765 - accuracy: 0.9465 - val_loss: 0.0654 - val_accuracy: 0.9820


Train for 1500 steps, validate for 375 steps
1500/1500 - 84s - loss: 0.2392 - accuracy: 0.9338 - val_loss: 0.0934 - val_accuracy: 0.9730


INFO:tensorflow:Oracle triggered exit
Train for 1875 steps, validate for 375 steps
1875/1875 - 17s - loss: 0.1584 - accuracy: 0.9514 - val_loss: 0.0453 - val_accuracy: 0.9861


In [8]:
clf.tuner.results_summary()

In [5]:
# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


[[7]
 [2]
 [1]
 ...
 [4]
 [5]
 [6]]


In [7]:
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test, verbose=2))

313/313 - 2s - loss: 0.0629 - accuracy: 0.9785
[0.06287696759891362, 0.9785]


## Validation Data

By default, AutoKeras use the last 20% of training data as validation data. As shown in the example below, you can use validation_split to specify the percentage.

In [ ]:
clf.fit(x_train,
        y_train,
        # Split the training data and use the last 15% as validation data.
        validation_split=0.15,epochs=3)

You can also use your own validation set instead of splitting it from the training data with validation_data.

In [ ]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(x_train,
        y_train,
        # Use your own validation set.
        validation_data=(x_val, y_val),epochs=3)

## Data Format

The AutoKeras ImageClassifier is quite flexible for the data format.

For the image, it accepts data formats both with and without the channel dimension. The images in the MNIST dataset do not have the channel dimension. Each image is a matrix with shape (28, 28). AutoKeras also accepts images of three dimensions with the channel dimension at last, e.g., (32, 32, 3), (28, 28, 1).

For the classification labels, AutoKeras accepts both plain labels, i.e. strings or integers, and one-hot encoded encoded labels, i.e. vectors of 0s and 1s.

So if you prepare your data in the following way, the ImageClassifier should still work.

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

# One-hot encode the labels.
import numpy as np
eye = np.eye(10)
y_train = eye[y_train]
y_test = eye[y_test]

print(x_train.shape) # (60000, 28, 28, 1)
print(y_train.shape) # (60000, 10)
print(y_train[:3])
# array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
#        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

We also support using tf.data.Dataset format for the training data. In this case, the images would have to be 3-dimentional. The labels have to be one-hot encoded for multi-class classification to be wrapped into tensorflow Dataset.

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.utils.data_utils import Sequence
train_set = tf.data.Dataset.from_tensor_slices(((x_train, ), (y_train, )))
test_set = tf.data.Dataset.from_tensor_slices(((x_test, ), (y_test, )))

clf = ak.ImageClassifier(max_trials=10)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))

## Reference
[ImageClassifier](/image_classifier),
[AutoModel](/auto_model/#automodel-class),
[ImageBlock](/block/#imageblock-class),
[Normalization](/preprocessor/#normalization-class),
[ImageAugmentation](/preprocessor/#image-augmentation-class),
[ResNetBlock](/block/#resnetblock-class),
[ImageInput](/node/#imageinput-class),
[ClassificationHead](/head/#classificationhead-class).